<a href="https://colab.research.google.com/github/sinamo101/Radioglaciology-Research/blob/main/NBA_Player_stat_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import csv
from google.colab import files
import io
import pandas as pd
from sklearn import datasets, linear_model



############################################################
# Optimization problem
teams = ['Boston Celtics', 
         'Phoenix Suns', 'Utah Jazz', 'Golden State Warriors', 'Memphis Grizzlies','Miami Heat',
'Dallas Mavericks',
'Milwaukee Bucks',
'Philadelphia 76ers',
'Toronto Raptors',
'Minnesota Timberwolves',
'Denver Nuggets',
'Cleveland Cavaliers',
'Atlanta Hawks',
'Brooklyn Nets',
'Charlotte Hornets',
'New York Knicks',
'Los Angeles Clippers',
'San Antonio Spurs',
'Chicago Bulls',
'New Orleans Pelicans',
'Los Angeles Lakers',
'Indiana Pacers',
'Washington Wizards',
'Sacramento Kings',
'Detroit Pistons',
'Orlando Magic',
'Oklahoma City Thunder',
'Houston Rockets',
'Portland Trail Blazers']
abbr = ['BOS', 'PHO', 'UTA', 'GSW', 'MEM', 'MIA', 'DAL', 'MIL', 'PHI', 'TOR', 'MIN', 'DEN', 'CLE', 'ATL', 'BRK', 'CHO', 'NYK', 'LAC', 'SAS', 'CHI', 'NOP', 'LAL', 'IND', 'WAS', 'SAC', 'DET', 'ORL', 'OKC', 'HOU', 'POR']

teams_to_abbr = {}
abbr_to_teams = {}
for i in range(len(teams)):
  teams_to_abbr[teams[i]] = abbr[i]
  abbr_to_teams[abbr[i]] = teams[i]

# getting 2021 stats ###########################
uploaded = files.upload()
holiday_2021 = pd.read_csv(io.BytesIO(uploaded['JHStats.csv']), usecols = ['G', 'Location', 'Opp', 'PTS'])

opponents_2021 = holiday_2021.Opp.values
opponents_2021 = opponents_2021.reshape(len(opponents_2021), 1)

uploaded = files.upload()
team_stats_2021 = pd.read_csv(io.BytesIO(uploaded['Team_Stats.csv']), usecols = ['Team', 'MOV', 'DRtg'])
teams = team_stats_2021.Team.values
defensive_ratings_2021 = team_stats_2021.DRtg.values
defensive_ratings_2021 = defensive_ratings_2021.reshape(len(defensive_ratings_2021), 1)

mov_ratings_2021 = team_stats_2021.MOV.values
mov_ratings_2021 = mov_ratings_2021.reshape(len(mov_ratings_2021), 1)

team_to_def_rating = {}
team_to_mov_rating = {}
for i in range(len(teams)):
  team_to_def_rating[teams[i]] = float(defensive_ratings_2021[i])
  team_to_mov_rating[teams[i]] = float(mov_ratings_2021[i])


holiday_2021=holiday_2021[holiday_2021["PTS"] != 'Inactive']
holiday_2021=holiday_2021[holiday_2021["PTS"] != 'Did Not Play']

points_2021 = holiday_2021.PTS.values
points_2021 = points_2021.reshape(len(points_2021), 1)

location_2021 = holiday_2021.Location.values
location_2021 = location_2021.reshape(len(location_2021), 1)
location_2021[location_2021 == "@"] = 1
location_2021[location_2021 != 1] = 0

game_num_2021 = holiday_2021.G.values
game_num_2021 = game_num_2021.reshape(len(game_num_2021), 1)

last_factors = []
for i in range(len(location_2021)):
  last_factors.append([int(location_2021[i]), int(game_num_2021[i]), float(team_to_def_rating[abbr_to_teams[opponents_2021[i][0]]]), float(team_to_mov_rating[abbr_to_teams[opponents_2021[i][0]]])])

# getting 2022 stats ######################
uploaded=files.upload()
holiday_2022 = pd.read_csv(io.BytesIO(uploaded['JHnew_games.csv']), usecols = ['G', 'Location', 'Opponent'])

opponents_2022 = holiday_2022.Opponent.values
opponents_2022 = opponents_2022.reshape(len(opponents_2022), 1)

location_2022 = holiday_2022.Location.values
location_2022 = location_2022.reshape(len(location_2022), 1)
location_2022[location_2022 == "@"] = 1
location_2022[location_2022 != 1] = 0

game_num_2022 = holiday_2022.G.values
game_num_2022 = game_num_2022.reshape(len(game_num_2022), 1)

new_factors = []

for i in range(len(location_2022)):
  new_factors.append([int(location_2022[i]), int(game_num_2022[i]), team_to_def_rating[opponents_2022[i][0]], team_to_mov_rating[opponents_2022[i][0]]
])
############

regr = linear_model.LinearRegression()

regr.fit(last_factors, points_2021)

last_predict = regr.predict(last_factors)
arr_last_predict = []

for i in range(len(last_predict)):
  arr_last_predict.append(last_predict[i][0])

new_predict = regr.predict(new_factors)

print("2021 predictions: " + str(last_predict))

print("2022 predictions: " + str(new_predict))

difference = []
new_points = []
for point in points_2021:
  new_points.append(int(point))

print(new_points)



difference = np.subtract(new_points, arr_last_predict)
abs_difference = [abs(ele) for ele in difference]

count = 0
for diff in abs_difference:
  if diff < 1:
    count += 1


print("% within 1 point: " + str(float(count) / len(abs_difference)))
print("Mean: " + str(np.mean(abs_difference)))
print("Median: " + str(np.median(abs_difference)))

Saving JHStats.csv to JHStats (43).csv


Saving Team_Stats.csv to Team_Stats (35).csv


Saving JHnew_games.csv to JHnew_games (22).csv
2021 predictions: [[16.43373814]
 [18.2921511 ]
 [17.11654761]
 [14.09002881]
 [16.93830812]
 [16.74862539]
 [17.69025162]
 [15.8390047 ]
 [18.31042525]
 [15.99719539]
 [18.22916556]
 [18.45012261]
 [20.86851435]
 [15.40584838]
 [16.45074451]
 [17.50857452]
 [17.15393902]
 [17.70812438]
 [16.85884793]
 [17.46254578]
 [14.88164718]
 [16.584994  ]
 [18.43973776]
 [19.82421799]
 [19.01245946]
 [19.91734956]
 [14.77657127]
 [19.19517519]
 [21.10594736]
 [15.30073926]
 [17.91098153]
 [19.84603955]
 [15.56358557]
 [20.25924934]
 [21.89296166]
 [18.54630854]
 [18.08525475]
 [18.23694204]
 [17.79016367]
 [19.73897572]
 [17.78875003]
 [17.51630973]
 [17.56287552]
 [21.83414549]
 [19.97180465]
 [17.40357313]
 [21.05758622]
 [18.23922486]
 [17.22803154]
 [20.68422371]
 [19.75856871]
 [19.46027054]
 [17.49190464]
 [16.44321113]
 [20.34063965]
 [18.35994177]
 [22.11878282]
 [16.1218547 ]
 [17.15876666]
 [20.51088911]
 [18.72006577]
 [18.44762547]
 [21.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

use points from previous year to predict points in the next game
